# Texas Heat-Related Mortality Estimates
This notebook accompanies the manuscript **"The True Cost of Heat: Evaluating Heat-Related Mortality Estimation Methods in Texas"**. It calculates heat-related deaths across Texas counties using the optimal temperature method, extreme heat method, and excess death method, based on ERA5 reanalysis data and mortality statistics.

**Code Author:** Jesse R. J. Rutt  
**For Questions about the code contact:** jesse_rutt@tamu.edu  
**Last Updated:** June 2025  

In [33]:
# Install missing packages and import libraries
import importlib
import subprocess
import sys

required_packages = {
    "pandas": "pandas",
    "numpy": "numpy",
    "xarray": "xarray",
    "netCDF4": "netCDF4",  # for full xarray NetCDF support
    "json": None  # built-in, no install needed
}

for module, package in required_packages.items():
    try:
        importlib.import_module(module)
    except ImportError:
        if package:
            print(f"Installing missing package: {package}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        else:
            print(f"Module '{module}' is built-in and should be available.")

# Re-import after potential install
import pandas as pd
import numpy as np
import xarray as xr
import json

print("All libraries are available and imported successfully.")

All libraries are available and imported successfully.


In [34]:
from pathlib import Path

# === Define base path ===
BASE_PATH = Path('/Users/jessesmac/Myprojects/data/coiled_output/')
# BASE_PATH = Path('')

# === Define all file paths ===
paths = {
    "County metadata (JSON)": BASE_PATH / 'combined_county_data.json',
    "ERA5 temperature data (NetCDF)": BASE_PATH / 'ERA5/griddedTXmeanTemp.nc',
    "Mortality file (CSV)": BASE_PATH / 'totals-by-day-texas.csv',
    "OTM reference (84th percentile)": BASE_PATH / 'ot_smt_mmt_deaths_84th.csv',
    "XHM reference (95th percentile)": BASE_PATH / 'ot_smt_mmt_deaths_95th.csv',
    "Output file path": BASE_PATH / 'HRD_CalculationsTable_1985-2006_OT_SMT.csv'
}

# === Validate paths ===
missing_files = [desc for desc, path in paths.items() if not path.exists() and 'Output' not in desc]

if missing_files:
    raise FileNotFoundError(f"The following required input files are missing:\n" +
                            "\n".join(f"- {desc}" for desc in missing_files))
else:
    print("All required input files exist.")

# Assign to variables for later use
county_data_path = paths["County metadata (JSON)"]
temperature_file = paths["ERA5 temperature data (NetCDF)"]
mortality_file = paths["Mortality file (CSV)"]
ot_84th_path = paths["OTM reference (84th percentile)"]
ot_95th_path = paths["XHM reference (95th percentile)"]
output_csv_path = paths["Output file path"]

All required input files exist.


In [35]:
# Define input and output paths
BASE_PATH = '/Users/jessesmac/Myprojects/data/coiled_output/'

county_data_path = BASE_PATH + 'combined_county_data.json'
temperature_file = BASE_PATH + 'ERA5/griddedTXmeanTemp.nc'
mortality_file = BASE_PATH + 'totals-by-day-texas.csv'
ot_84th_path = BASE_PATH + 'ot_smt_mmt_deaths_84th.csv'
ot_95th_path = BASE_PATH + 'ot_smt_mmt_deaths_95th.csv'
output_csv_path = BASE_PATH + 'HRD_CalculationsTable_1985-2006_OT_SMT.csv'

print("All file paths are valid.")

All file paths are valid.


# Heat-Related Mortality Estimates in Texas By County

In [36]:
# Load all input datasets
with open(county_data_path) as f:
    county_data = json.load(f)

ot_84th_df = pd.read_csv(ot_84th_path)
ot_95th_df = pd.read_csv(ot_95th_path)
population_data = pd.read_csv(mortality_file).groupby('county')['population'].mean().to_dict()

print(f"Loaded {len(county_data)} counties.")

Loaded 254 counties.


In [37]:
def get_ot_smt_mmt(county, df):
    """
    Retrieve OT, SMT, and baseline MMT deaths for a given county.
    """
    try:
        row = df[df['county'] == county]
        return row['OT'].values[0], row['SMT'].values[0], row['mmt_deaths'].values[0]
    except IndexError:
        return None, None, 0

def calculate_hrd(county, start_year, end_year, lat, lon, baseline_df):
    """
    Estimate total heat-related deaths from daily temperature time series.
    """
    dataset = xr.open_dataset(temperature_file).squeeze()
    total_hrd = 0
    ot, smt, mmt_deaths = get_ot_smt_mmt(county, baseline_df)
    if ot is None:
        return 0

    for year in range(start_year, end_year + 1):
        temps = dataset.sel(lat=lat, lon=lon, method='nearest').sel(time=slice(f'{year}-01-01', f'{year}-12-31')).to_dataframe()
        temps['t2m'] -= 273.15  # Kelvin to Celsius

        rr_func = lambda t: 1 - 0.0014 * (smt - 30.9) * (t - ot) ** 2 + 0.005 * (smt - 26.7) * (t - ot)
        temps['rr'] = rr_func(temps['t2m'])
        temps['heatDeaths'] = (temps['rr'] - 1) * mmt_deaths
        temps.loc[temps['t2m'] < ot, 'heatDeaths'] = 0

        total_hrd += temps['heatDeaths'].sum()

    return total_hrd

def calculate_excess_hrd(county, lat, lon, df):
    return calculate_hrd(county, 2010, 2023, lat, lon, df) - calculate_hrd(county, 1950, 1963, lat, lon, df)

def format_population(pop):
    return round(pop / 1000, 1)


In [38]:
results = {
    'County': [],
    'Population (Thousands)': [],
    'OTM HRD': [],
    'XHM HRD': [],
    'Excess OTM': [],
    'Excess XHM': []
}

for county, info in county_data.items():
    lat, lon = info['latitude'], info['longitude']
    population = population_data.get(county.upper(), 0)
    if population == 0:
        continue

    otm = calculate_hrd(county, 2010, 2023, lat, lon, ot_84th_df)
    xhm = calculate_hrd(county, 2010, 2023, lat, lon, ot_95th_df)
    excess_otm = calculate_excess_hrd(county, lat, lon, ot_84th_df)
    excess_xhm = calculate_excess_hrd(county, lat, lon, ot_95th_df)

    results['County'].append(county)
    results['Population (Thousands)'].append(format_population(population))
    results['OTM HRD'].append(round(otm, 1))
    results['XHM HRD'].append(round(xhm, 1))
    results['Excess OTM'].append(round(excess_otm, 1))
    results['Excess XHM'].append(round(excess_xhm, 1))

In [39]:
results_df = pd.DataFrame(results).sort_values(by='Population (Thousands)', ascending=False)
results_df.to_csv(output_csv_path, index=False)

print("Top 10 Counties by Population and HRD:")
results_df.head(10)

Top 10 Counties by Population and HRD:


,County,Population (Thousands),OTM HRD,XHM HRD,Excess OTM,Excess XHM
1,HARRIS,4629.4,1584.3,400.7,1000.8,322.8
2,DALLAS,2583.9,1768.0,464.4,918.3,351.2
3,TARRANT,2050.9,1629.8,401.3,787.1,294.7
0,BEXAR,1955.0,953.5,183.5,557.8,138.8
4,TRAVIS,1234.2,512.1,90.6,306.0,73.0
5,COLLIN,996.0,559.5,144.0,281.6,105.6
7,HIDALGO,856.5,203.9,43.7,151.3,41.7
6,DENTON,848.7,411.2,104.7,194.7,74.9
59,ELPASO,844.6,560.9,72.7,339.1,52.7
64,FORTBEND,774.1,167.6,39.5,109.9,33.6


# Heat-Related Mortality Estimates in Texas By Year

In [40]:
# Define compact per-year HRD and excess HRD calculations using existing functions/data
def calculate_total_hrd_by_year(year, df):
    total = 0
    for county, loc in county_data.items():
        lat, lon = loc['latitude'], loc['longitude']
        ot, smt, mmt = get_ot_smt_mmt(county, df)
        if ot is None:
            continue
        temps = xr.open_dataset(temperature_file).squeeze().sel(lat=lat, lon=lon, method='nearest')
        temps = temps.sel(time=slice(f'{year}-01-01', f'{year}-12-31')).to_dataframe()
        temps['t2m'] -= 273.15
        rr = lambda t: 1 - 0.0014 * (smt - 30.9) * (t - ot)**2 + 0.005 * (smt - 26.7) * (t - ot)
        temps['rr'] = rr(temps['t2m'])
        temps['heatDeaths'] = (temps['rr'] - 1) * mmt
        temps.loc[temps['t2m'] < ot, 'heatDeaths'] = 0
        total += temps['heatDeaths'].sum()
    return total

def calculate_excess_by_year(year, df):
    return calculate_total_hrd_by_year(year, df) - calculate_total_hrd_by_year(1950 + (year - 2010), df)

In [41]:
# Predefined official death counts
official_deaths = {
    2010: 115, 2011: 218, 2012: 99, 2013: 88, 2014: 82, 2015: 85,
    2016: 141, 2017: 85, 2018: 137, 2019: 159, 2020: 141, 2021: 241,
    2022: 419, 2023: 563
}

In [43]:
# Compile yearly results
results_by_year = {
    "Year": [], "OTM HRD": [], "XHM HRD": [],
    "Excess OTM": [], "Excess XHM": [], "Official Deaths": []
}

for year in range(2010, 2024):
    otm = calculate_total_hrd_by_year(year, ot_84th_df)
    xhm = calculate_total_hrd_by_year(year, ot_95th_df)
    results_by_year["Year"].append(year)
    results_by_year["OTM HRD"].append(round(otm, 1))
    results_by_year["XHM HRD"].append(round(xhm, 1))
    results_by_year["Excess OTM"].append(round(calculate_excess_by_year(year, ot_84th_df), 1))
    results_by_year["Excess XHM"].append(round(calculate_excess_by_year(year, ot_95th_df), 1))
    results_by_year["Official Deaths"].append(official_deaths[year])

# Save results
df_year = pd.DataFrame(results_by_year)
year_csv_path = BASE_PATH / 'HRD_CalculationsTable_ByYear_2010-2023.csv'
df_year.to_csv(year_csv_path, index=False)

print(f"Yearly HRD summary saved to: {year_csv_path}")
df_year.head()

TypeError: unsupported operand type(s) for /: 'str' and 'str'